In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

#URL of the website
URL = "https://www.redbus.in/online-booking/jkrtc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

#Function to scrape Bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

#Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5) #Allow the page to load
    
        #Click the "View Buses" button if it exists
        try:
            view_buses_button =  WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'button'))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)

            #Scroll down to view all Bus details
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

           #Find Bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travelsName___950ec8")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "busType___675d0a")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "boardingTime___a78ae0")
            duration_elements = driver.find_elements(By.CLASS_NAME, "duration___b3a515")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "droppingTime___c814da")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating___f7ba20')]")
            price_elements = driver.find_elements(By.CLASS_NAME, "finalFare___057afc")

            # Use XPath to handle both seat availability classes 
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'totalSeats___53250b')]")
            
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

#Listing Bus details
all_bus_details = []

#Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  #There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
             # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

#Scrape routes and details from all pages
scrape_all_pages()

#Convert the list of dictionary into Dataframe
df = pd.DataFrame(all_bus_details)

#save Dataframe to CSV 
df.to_csv('Jammu_Kashmir_details.csv', index=False)

#Close the driver
driver.quit()

NameError: name 'driver' is not defined

In [33]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,JKRTC - 172597,Volvo A/C Seater (2+2),13:00,24h,13:00,4.3,"₹2,569",0
1,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,20h 40m,18:15,4,"₹2,970",0
2,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,20h 10m,16:10,3.9,"₹2,581",0
3,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Big Bus,A/C Seater / Sleeper (2+2),21:15,18h 15m,15:30,3.2,"₹2,850",0
4,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Lavish Travels,A/C Seater / Sleeper (2+2),20:30,18h 15m,14:45,3,"₹2,799",0
...,...,...,...,...,...,...,...,...,...,...
96,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),20:50,7h 30m,04:20,4.3,₹831,0
97,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),16:30,8h 10m,00:40,4.2,₹770,0
98,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Jamna Travels-Jammu,AshokLeyland Stile A/C,21:00,7h 30m,04:30,3.9,₹481,0
99,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...,Harikesh Tour N Travels,A/C Sleeper (2+1),21:10,7h 35m,04:45,4.2,₹559,0
